# 03 - Adversarial Attacks (FGSM)

**Goal:** Generate adversarial examples using FGSM and extract features for TDA analysis

### Overview
In this notebook, we will:
1. Build a simple CIFAR-10 classifier
2. Generate FGSM adversarial examples
3. Visualize clean vs adversarial images
4. Extract features from adversarial images
5. Compare clean vs adversarial feature distributions

**Why FGSM?**
- Fast and simple adversarial attack
- Creates imperceptible perturbations
- Good starting point for adversarial robustness analysis

### Setup and Imports

In [2]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from src.data.cifar10 import load_cifar10
from src.models.feature_extractor import FeatureExtractor, DimensionalityReducer
from src.attacks.fgsm import FGSMAttack, generate_fgsm_dataset

np.random.seed(42)       # For reproducibility
tf.random.set_seed(42)

print("Successfully imported everything")

Successfully imported everything


### Load the CIFAR-10 Dataset

We'll be using a subset of 5,000 training samples for faster experimentation.

In [5]:
(training_images, training_labels), (validation_images, validation_labels), (testing_images, testing_labels), class_names = load_cifar10(validation_split=0.1)

training_images_subset = training_images[:5000]
training_labels_subset = training_labels[:5000]

# Verify:
print(f"\nUsing {len(training_images_subset)} samples")
print(f"Image shape: {training_images_subset.shape}")
print(f"Label shape: {training_labels_subset.shape}")


Final splits:
Training:   45000 samples
Validation: 5000 samples
Testing:    10000 samples

Using 5000 samples
Image shape: (5000, 32, 32, 3)
Label shape: (5000,)


### Build the CIFAR-10 Classifier

We'll create a simple CNN to classify CIFAR-10 images. This will be our "victim model" that we'll attack later